In [26]:
p=57896044618658097711785492504343953926634992332820282019728792003956565153471
K = GF(p)
q=14474011154664524427946373126085988481777980136002577561989638920376330856411
aWeier = K(14474011154664524427946373126085988481658748083205070504932198000989141288364)
bWeier = K(14474011154664524427946373126085988481658748083205070504932198000989141288365)
xWeier = K(1)
yWeier = K(1370184815209153778412612456839413869416711536549829925085495630500881379458)
zWeier = K(1)
alpha = K(-3).sqrt()
gamma = K(2)

In [27]:
additions = 0
multiplications=0

In [28]:
def add(a, b):
    global additions
    additions += 1
    return a+b
def mul(a, b):
    global multiplications
    multiplications += 1
    return a*b

In [100]:
def addTwoPointsWeierstrass(x1, y1, z1, x2, y2, z2, a, b, ):
    if(x1==x2 and y1==y2):
        w=add(mul(a,mul(z1,z1)),mul(3, mul(x1,x1)))
        s=mul(y1,z1)
        ss=mul(s, s)
        sss = mul(s, ss)
        r=mul(y1, s)
        b=mul(x1, r)
        b4=mul(4, b)
        h=add(mul(w, w), -add(b4, b4))
        x3=mul(h, s)
        x3=add(x3,x3)
        y3=add(mul(w, add(b4, -h)),-mul(8, mul(r,r)))
        z3=mul(8, sss)
    else:
        y1z2=mul(y1,z2)
        x1z2=mul(x1,z2)
        z1z2=mul(z1,z2)
        u=add(mul(y2,z1),-y1z2)
        uu=mul(u,u)
        v=add(mul(x2,z1),-x1z2)
        vv=mul(v,v)
        vvv=mul(v, vv)
        R=mul(vv, x1z2)
        A=add(mul(uu, z1z2),-add(vvv, add(R, R)))
        x3=mul(v, A)
        y3=add(mul(u, add(R, -A)), -mul(vvv, y1z2))
        z3=mul(vvv, z1z2)
    return x3, y3, z3
              

In [101]:
addTwoPointsWeierstrass(aWeier, bWeier, xWeier, yWeier, zWeier, xWeier, yWeier, zWeier)

(20603904670834645468580229852244388859270111775432385519915925046572775578253,
 8777153777661139624351339573670995835579111346988115026467202711985232462330,
 5941996246825723184926951811657656609145688492356356955219264565971840413230)

In [31]:
multiplications

14

In [32]:
def convertWeierStrassToMontgomeryCoeffs(a, b):
    R.<x> = PolynomialRing(K, 'x')
    equation = x^3 + a*x + b
    roots = equation.roots(multiplicities=False)
    theta = roots[0]
    gamma = sqrt(3*theta^2+a)
    mu=3*theta/gamma
    return gamma, mu
def convertWeierStrassToMontgomeryPoints(a, b, x, y):
    R.<x> = PolynomialRing(K, 'x')
    equation = x^3 + a*x + b
    roots = equation.roots(multiplicities=False)
    theta = roots[0]
    gamma = sqrt(3*theta^2+a)
    xMont = (xWeier-theta)/gamma
    yMont = yWeier / gamma^2
    return xMont, yMont
def convertMontgomeryToWeierstrassCoeffs(gamma, mu):
    theta = gamma * mu / 3
    a = gamma^2 - 3*theta^2
    b = -(theta^3 + a*theta)
    return a, b

def convertMontgomeryToWeierstrassPoints(gamma, mu, x, y):
    theta = gamma * mu / 3
    xWeier = gamma * x + theta
    yWeier = gamma^2 * y
    return xWeier, yWeier

In [33]:
gammaMont, muMont = convertWeierStrassToMontgomeryCoeffs(aWeier, bWeier)
xMont, yMont = convertWeierStrassToMontgomeryPoints(aWeier, bWeier, xWeier, yWeier)
zMont = K(1)

In [34]:
xWeier, yWeier, convertMontgomeryToWeierstrassPoints(gammaMont, muMont, xMont, yMont)

(1,
 1370184815209153778412612456839413869416711536549829925085495630500881379458,
 (1,
  1370184815209153778412612456839413869416711536549829925085495630500881379458))

In [35]:
def convertMontgomeryToEdwardsCoeffs(gamma, mu):
    e = (mu + 2)/gamma
    d = (mu - 2)/gamma
    return e, d

def convertMontgomeryToEdwardsPoints(gamma, mu, x, y):
    xEdwards = x/y
    yEdwards = (x-1)/(x+1)
    return xEdwards, yEdwards

def convertEdwardsToMontgomeryCoeffs(e, d):
    mu = 2 * (e + d) / (e - d)
    gamma = 4 / (e - d)
    return gamma, mu

def convertEdwardsToMontgomeryPoints(e, d, x, y):
    xMont = (1 + y) / (1 - y)
    yMont = xMont/x
    return xMont, yMont


In [36]:
eEdwards, dEdwards = convertMontgomeryToEdwardsCoeffs(gammaMont, muMont)
xEdwards, yEdwards = convertMontgomeryToEdwardsPoints(gammaMont, muMont, xMont, yMont)
zEdwards = K(1)

In [37]:
def convertWeierstrassToHadanoCoeffs(a, b, gamma):
    R.<x> = PolynomialRing(K, 'x')
    equation = x^3 + a*x + b
    roots = equation.roots(multiplicities=False)
    theta = roots[0]
    newA = 3*theta*gamma
    newB = (3*theta^2 + a) * gamma^2
    return newA, newB

def convertWeierstrassToHadanoPoints(a, b, gamma, xWeier, yWeier):
    R.<x> = PolynomialRing(K, 'x')
    equation = x^3 + a*x + b
    roots = equation.roots(multiplicities=False)
    theta = roots[0]
    xHadano = gamma * (xWeier-theta)
    yHadano = (gamma.sqrt() ** 3) * yWeier
    return xHadano, yHadano

# def convertHadanoToWeierstrassCoeffs(a, b, gamma):
    
#     return 

# def convertEdwardsToMontgomeryPoints(e, d, x, y):
#     xMont = (1 + y) / (1 - y)
#     yMont = xMont/x
#     return xMont, yMont


In [38]:
xHadano, yHadano = convertWeierstrassToHadanoPoints(aWeier, bWeier, gamma, xWeier, yWeier)
aHadano, bHadano = convertWeierstrassToHadanoCoeffs(aWeier, bWeier, gamma)

In [39]:
zHadano = K(1)

In [40]:
def addTwoPointsEdwards(x1, y1, z1, x2, y2, z2, e, d):
    if(x1==x2 and y1==y2 and z1 == z2):
        B = add(x1,y1)
        B = mul(B, B)
        C = mul(x1, x1)
        D = mul(y1, y1)
        E = mul(e, C)
        F = add(E, D)
        H = mul(z1, z1)
        J = add(F, -add(H, H))
        x3 = mul(add(B, -add(C, D)), J)
        y3 = mul(F, add(E, -D))
        z3 = mul(F, J)
    else:
        A = mul(z1, z2)
        B = mul(A, A)
        C = mul(x1, x2)
        D = mul(y1, y2)
        E = mul(d, mul(C, D))
        F = add(B, -E)
        G = add(B, E)
        x3 = mul(mul(A, F), add(mul(add(x1, y1), add(x2, y2)), -add(C, D)))
        y3 = mul(mul(A, G), add(D, -mul(e, C)))
        z3 = mul(F, G)
    return x3, y3, z3
              

In [41]:
def addTwoPointsHadano(x1, y1, z1, x2, y2, z2, a, b):
    if(x1==x2 and y1==y2 and z1 == z2):
        y1z1 = mul(y1, z1)
        y12z1 = mul(y1z1, y1)
        az1 = mul(a, z1)
        x12 = add(x1, x1)
        ax12z1 = mul(az1, x12)
        z12 = mul(z1, z1)
        bz12 = mul(b, z12)
        delta = add(add(mul(3, mul(x1, x1)), ax12z1), bz12)
        delta2 = mul(delta, delta)
        y12z14 = mul(4, y12z1)
        y13z12 = mul(y12z14, y1z1)
        az1p2x = add(az1, x12)
        z3 = mul(y13z12, add(z1, z1))
        x3 = add(-mul(y12z14, az1p2x), delta2)
        y3 = add(mul(y12z14, add(-add(y12z1, y12z1),
                                 mul(x1, delta))), -mul(x3, delta))
        x3 = mul(x3, add(y1z1,y1z1)) 
    else:
        z1z2 = mul(z1, z2)
        y2z1 = mul(y2, z1)
        y1z2 = mul(y1, z2)
        x2z1 = mul(x2, z1)
        x1z2 = mul(x1, z2)
        llambda = add(y2z1, -y1z2)
        delta = add(x2z1, -x1z2)
        llambda2 = mul(llambda, llambda)
        delta2 = mul(delta, delta)
        deltap = add(x2z1, x1z2)
        adelta2 = mul(a, delta2)
        x3 = add(mul(z1z2, add(llambda2, -adelta2)), -mul(delta2, deltap))
        z3 = mul(mul(z1z2, delta2), delta)
        y3 = add(mul(mul(z2, delta2), add(-mul(y1, delta), 
                            mul(llambda, x1))), -mul(llambda, x3))
        x3 = mul(x3, delta)        
    return x3, y3, z3
              

In [42]:
def endomorphismHadano(x, y, z, alpha):
    xx1 = add(x, K(1))
    xx1xx1 = mul(xx1, xx1)
    z13 = mul(K(3), z)
    xx3 = add(x, -z13)
    delta = mul(alpha, xx1)
    znew = mul(mul(z13,delta), xx1xx1)
    xnew = -mul(mul(delta, x), mul(xx3, xx3))
    x1z13 = mul(x, z13)
    ynew = mul(mul(xx3, add(add(mul(x, x), 
                                add(x1z13, x1z13)), mul(z13, z))), y)
    return xnew, ynew, znew
              

In [102]:
additions = 0
multiplications=0
tmpxHadano, tmpyHadano, tmpzHadano = addTwoPointsHadano(xHadano, yHadano, zHadano, xHadano, yHadano, zHadano, aHadano, bHadano)
additions, multiplications

(10, 17)

In [106]:
additions = 0
multiplications=0
tmpxWeier, tmpyWeier, tmpzWeier = addTwoPointsWeierstrass(xWeier, yWeier, zWeier, xWeier, yWeier, zWeier, aWeier, bWeier)
additions, multiplications

(6, 16)

In [107]:
additions = 0
multiplications=0
tmpxWeier, tmpyWeier, tmpzWeier = addTwoPointsWeierstrass(xWeier, yWeier, zWeier, tmpxWeier, tmpyWeier, tmpzWeier, aWeier, bWeier)
additions, multiplications

(7, 14)

In [108]:
additions = 0
multiplications=0
tmpxHadano, tmpyHadano, tmpzHadano = addTwoPointsHadano(xHadano, yHadano, zHadano, tmpxHadano, tmpyHadano, tmpzHadano, aHadano, bHadano)
additions, multiplications

(7, 18)

In [65]:
from timeit import default_timer as timer
def multiplyPointStandardVersion(Px, Py, Pz, k, a, b, func):
    binary = Integer(k).digits(base=2)[::-1]
    resx, resy, resz = 0, 1, 0
    px, py, pz = Px, Py, Pz

    for digit in binary:
        if digit == 1:
            resx, resy, resz = func(resx, resy, resz, px, py, pz, a, b)
        px, py, pz = func(px, py, pz, px, py, pz, a, b)

    return resx, resy, resz


def multiplyPointEndoVersion(Px, Py, Pz, k, a, b, func1, func2, AComplex, p):
    binary = calculateW(AComplex, k)
    resx, resy, resz = 0, 1, 0
    px, py, pz = Px, Py, Pz
    before = timer()

    for digit in binary:
        if digit == 1:
            resx, resy, resz = func1(resx, resy, resz, px, py, pz, a, b)
        if digit == -1:
            tmpy = add(p, -py)
            resx, resy, resz = func1(resx, resy, resz, px, tmpy, pz, a, b)
        px, py, pz = func2(px, py, pz, alpha)
        
    after = timer()
    return resx, resy, resz, after - before
def tr(a):
    return (a + conjugate(a)).real().n(digits=2048)

def nA(a):
    return (conjugate(a) * a).real().n(digits=2048)

def calculateW(a, k):
    na = nA(a)
    trA = tr(a)
    sigmaA = round(na) % 2
    nAa = (na - sigmaA) / 2
    s0 = k
    s1 = 0
    wArray = [0] * (round(2 * log(k) / log(na)) + 5)
    w = 0
    while True:
        q = quotient(s0, na)
        xW = s0 % na
        if xW > nAa:
            xW = xW - na
            q += 1
        wArray[w] = round(xW.real())
        s0 = round((q * trA + s1).real())
        s1 = -q
        w += 1
        if abs(s0) < nAa and s1 == 0:
            break

    return wArray

In [72]:
calculateW(sqrt(-7), 9)

[2, 0, 0, 0, 0, 0, 0]

In [71]:
calculateW(sqrt(-3), 9)

[0, 0, 0, 0, 1, 0, 0, 0, 0]

In [59]:
def montgomeryAddPoints(x1, z1, x2, z2, mx1, mz1):
    a = add(x1,-z1); 
    b = add(x1, z1); 
    c = add(x2,-z2); 
    d = add(x2, z2)
    e = mul(a,d); 
    f = mul(b, c);
    epf = add(e, f)
    emf = add(e, -f)
    x3 = mul(mz1, mul(epf, epf))
    z3 = mul(mx1, mul(emf, emf))
    return [x3, z3]

def montgomeryDoublePoint(x, z, C):
    a=add(x, z);
    b=add(x, -z);
    c=mul(a, a); 
    d=mul(b, b); 
    e=add(c, -d)
    x3 = mul(c, d)
    z3 = mul(e, add(d, mul(C, e)))
    return [x3, z3]
    
def montgomeryLadder(x, z, n, mu):
    C = (mu + 2) / 4
    Q = [[x, z], montgomeryDoublePoint(x, z, C)]
    b = Integer(n).digits(base=2)
    for i in range(len(b)-2,-1,-1):
        Q[Integer(1)-b[i]] = montgomeryAddPoints(Q[1][0], Q[1][1], Q[0][0], Q[0][1], x, z)
        Q[b[i]] = montgomeryDoublePoint(Q[b[i]][0], Q[b[i]][1],C)
    return Q[0][0], Q[0][1]

In [60]:
floor(log(q,10))

76

In [109]:
import random
from timeit import default_timer as timer
results = []  # Список для хранения результатов

for i in range(2, 75):
    j = round(log(10**i, 2)*2)  # Вычисляем j
    numbers = [random.randint(10**i, 10**(i+1) - 1) for _ in range(j)]  # Генерируем j случайных чисел
    for number in numbers:
        additions = 0  # Счетчик сложений
        multiplications = 0  # Счетчик умножений


        start = timer()
        multiplyPointStandardVersion(xWeier, yWeier, zWeier, number, aWeier, bWeier, addTwoPointsWeierstrass)
        end = timer()
        weierTime = end - start
        
        weierAdditions = additions
        weierMultiplications = multiplications
        
        additions = 0
        multiplications = 0

        start = timer()
        montgomeryLadder(xMont, zMont, number, muMont) 
        end = timer()
        montTime = end - start
        
        
        montAdditions = additions
        montMultiplications = multiplications
        
        additions = 0
        multiplications = 0

        start = timer()
        multiplyPointStandardVersion(xHadano, yHadano, zHadano, number, aHadano, bHadano, addTwoPointsHadano)
        end = timer()
        hadanoTime = end - start
        
        hadanoAdditions = additions
        hadanoMultiplications = multiplications
        
                
        additions = 0
        multiplications = 0

        start = timer()
        multiplyPointStandardVersion(xEdwards, yEdwards, zEdwards, number, eEdwards, dEdwards, addTwoPointsEdwards)
        end = timer()
        edwardsTime = end - start
        
        edwardsAdditions = additions
        edwardsMultiplications = multiplications
        
                
        additions = 0
        multiplications = 0

        _, _, _, endoTime = multiplyPointEndoVersion(xHadano, yHadano, zHadano, number, aHadano, bHadano, addTwoPointsHadano, endomorphismHadano, sqrt(-3), p)
        
        endoAddition = additions
        endoMultiplications = multiplications

        results.append({'number': str(number), "endoMultiplications": str(endoMultiplications), "endoAdditions": str(endoAddition),
                       "weierAdditions": str(weierAdditions), "weierMultiplications": str(weierMultiplications), 
                       "montAdditions": str(montAdditions), "montMultiplications": str(montMultiplications),
                       "hadanoAdditions": str(hadanoAdditions), "hadanoMultiplications": str(hadanoMultiplications),
                       "edwardsAdditions": str(edwardsAdditions), "edwardsMultiplications": str(edwardsMultiplications),
                       "weierTime": weierTime, "endoTime": endoTime, "edwardsTime": edwardsTime, "montTime": montTime, "hadanoTime": hadanoTime})        

In [110]:
len(results)

18430

In [111]:
import json
with open('resultsWithEndo.json', 'w') as f:
    json.dump(results, f)

In [374]:
results

[{'number': 959,
  'endoMultiplications': 311,
  'endoAddition': 123,
  'weierAdditions': 123,
  'weierMultiplications': 286,
  'montAdditions': 94,
  'montMultiplications': 104,
  'hadanoAdditions': 163,
  'hadanoMultiplications': 332,
  'edwardsAdditions': 133},
 {'number': 936,
  'endoMultiplications': 293,
  'endoAddition': 115,
  'weierAdditions': 95,
  'weierMultiplications': 230,
  'montAdditions': 94,
  'montMultiplications': 104,
  'hadanoAdditions': 135,
  'hadanoMultiplications': 260,
  'edwardsAdditions': 105},
 {'number': 831,
  'endoMultiplications': 311,
  'endoAddition': 123,
  'weierAdditions': 116,
  'weierMultiplications': 272,
  'montAdditions': 94,
  'montMultiplications': 104,
  'hadanoAdditions': 156,
  'hadanoMultiplications': 314,
  'edwardsAdditions': 126},
 {'number': 828,
  'endoMultiplications': 293,
  'endoAddition': 115,
  'weierAdditions': 102,
  'weierMultiplications': 244,
  'montAdditions': 94,
  'montMultiplications': 104,
  'hadanoAdditions': 142,
 

In [369]:
len("6043422556832972882497885954803044486949960629401906949314")

58